In [10]:
# Import dependancies
import pandas as pd
import datetime as dt
from datetime import timedelta
from sqlalchemy import create_engine

In [11]:
#Import Telus "News" data into a Panadas DataFrame for Transformation

news_file = "new.csv"
news_df = pd.read_csv(news_file)
#news_df['date']=pd.to_datetime(news_df['date'])
news_df.head()

,Date,title,desc
0,2019-11-17,Tesla casually updates Model 3 to be devilishl...,Tesla Model 3 is advertised as an affordable e...
1,2019-11-18,Four must-haves for the Tesla ‘Cybertruck’ all...,Tesla is set to unveil its pickup this week an...
2,2019-11-19,Four must-haves for the Tesla ‘Cybertruck’ all...,Tesla is set to unveil its pickup this week an...
3,2019-11-20,What the Heck Is the Tesla Cybertruck? Let's W...,Eager to challenge the dominance of Ford’s F-1...
4,2019-11-21,Tesla all-electric ATV makes a surprise debut ...,Tesla CEO Elon Musk definitely didn’t have the...


In [12]:
# Concatenate all news info into a single column entry (i.e. concatenate the title string and the desc string)

news_df['contents'] = news_df['title'] + news_df['desc']

news_df.head()

,Date,title,desc,contents
0,2019-11-17,Tesla casually updates Model 3 to be devilishl...,Tesla Model 3 is advertised as an affordable e...,Tesla casually updates Model 3 to be devilishl...
1,2019-11-18,Four must-haves for the Tesla ‘Cybertruck’ all...,Tesla is set to unveil its pickup this week an...,Four must-haves for the Tesla ‘Cybertruck’ all...
2,2019-11-19,Four must-haves for the Tesla ‘Cybertruck’ all...,Tesla is set to unveil its pickup this week an...,Four must-haves for the Tesla ‘Cybertruck’ all...
3,2019-11-20,What the Heck Is the Tesla Cybertruck? Let's W...,Eager to challenge the dominance of Ford’s F-1...,What the Heck Is the Tesla Cybertruck? Let's W...
4,2019-11-21,Tesla all-electric ATV makes a surprise debut ...,Tesla CEO Elon Musk definitely didn’t have the...,Tesla all-electric ATV makes a surprise debut ...


In [13]:
#Import Tesla "stock" data into a Panadas DataFrame for Transformation

stocks_file = "stock.csv"
stocks_df = pd.read_csv(stocks_file)
#stocks_df['date']=pd.to_datetime(stocks_df['date'])
stocks_df.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2019-11-18,353.149994,346.100006,352.920013,349.989990,4400400,349.989990
1,2019-11-19,359.989990,347.799988,351.750000,359.519989,7724800,359.519989
2,2019-11-20,361.200012,349.570007,360.000000,352.220001,6725100,352.220001
3,2019-11-21,360.839996,354.000000,354.510010,354.829987,6110000,354.829987
4,2019-11-22,341.000000,330.000000,340.160004,333.040009,16870600,333.040009


In [14]:
#Create a date dimension table so we can create a date mapping to address the null values created in the above merge of the News df with the Stocks df
start_date ='2019-11-17'
end_date ='2019-12-16'

def date_table(start,end):
    df = pd.DataFrame({"date":pd.date_range(start, end)})
    df["day"] = df.date.dt.weekday_name
    return df

df = date_table(start_date, end_date)
df.head()

,date,day
0,2019-11-17,Sunday
1,2019-11-18,Monday
2,2019-11-19,Tuesday
3,2019-11-20,Wednesday
4,2019-11-21,Thursday


In [15]:
# Use logic to create a new column called "Next_Mon" to create a date mapping for weekend days (Sat and Sun) to be mapped to the next Monday

new_data = []

for index, row in df.iterrows():
    if row['day'] =='Saturday':
        nm = pd.to_datetime(row['date'])+ timedelta(days=2)
        row['next_mon'] = nm.strftime("%Y-%m-%d")
        new_row = [row['date'],row['day'], row['next_mon']]
    elif row['day'] == 'Sunday':
        nm = pd.to_datetime(row['date']) + timedelta(days=1)
        row['next_mon'] = nm.strftime("%Y-%m-%d")
        new_row = [row['date'],row['day'], row['next_mon']]
    else:
        nm = pd.to_datetime(row['date'])
        row['next_mon'] = nm.strftime("%Y-%m-%d")
        new_row = [row['date'],row['day'], row['next_mon']]
    new_data.append(new_row)


In [16]:
# Renamed columns of the new date dimension table

df2 = pd.DataFrame(new_data)
df2.columns=['date', 'day', 'next_mon']
df2.head()
df2['date'] = df2['date'].astype(str)
df2.head()

,date,day,next_mon
0,2019-11-17,Sunday,2019-11-18
1,2019-11-18,Monday,2019-11-18
2,2019-11-19,Tuesday,2019-11-19
3,2019-11-20,Wednesday,2019-11-20
4,2019-11-21,Thursday,2019-11-21


In [17]:
# Merge the date dimension table with the News dataframe to create a new dataframe called "ab"

ab = pd.merge(df2,news_df, left_on='date', right_on='Date', how='left')
ab.head()
#print(df2.columns, news_df.columns)

,date,day,next_mon,Date,title,desc,contents
0,2019-11-17,Sunday,2019-11-18,2019-11-17,Tesla casually updates Model 3 to be devilishl...,Tesla Model 3 is advertised as an affordable e...,Tesla casually updates Model 3 to be devilishl...
1,2019-11-18,Monday,2019-11-18,2019-11-18,Four must-haves for the Tesla ‘Cybertruck’ all...,Tesla is set to unveil its pickup this week an...,Four must-haves for the Tesla ‘Cybertruck’ all...
2,2019-11-19,Tuesday,2019-11-19,2019-11-19,Four must-haves for the Tesla ‘Cybertruck’ all...,Tesla is set to unveil its pickup this week an...,Four must-haves for the Tesla ‘Cybertruck’ all...
3,2019-11-20,Wednesday,2019-11-20,2019-11-20,What the Heck Is the Tesla Cybertruck? Let's W...,Eager to challenge the dominance of Ford’s F-1...,What the Heck Is the Tesla Cybertruck? Let's W...
4,2019-11-21,Thursday,2019-11-21,2019-11-21,Tesla all-electric ATV makes a surprise debut ...,Tesla CEO Elon Musk definitely didn’t have the...,Tesla all-electric ATV makes a surprise debut ...


In [18]:
newab= ab[['next_mon','contents']]

newab.head()

,next_mon,contents
0,2019-11-18,Tesla casually updates Model 3 to be devilishl...
1,2019-11-18,Four must-haves for the Tesla ‘Cybertruck’ all...
2,2019-11-19,Four must-haves for the Tesla ‘Cybertruck’ all...
3,2019-11-20,What the Heck Is the Tesla Cybertruck? Let's W...
4,2019-11-21,Tesla all-electric ATV makes a surprise debut ...


In [19]:
# Merge rows with same Date and stock info but different contents by joining the content cels together (concatenation)

df = newab.groupby('next_mon').agg({
     'contents':' ' .join}).reset_index()

In [20]:
df.head()

,next_mon,contents
0,2019-11-18,Tesla casually updates Model 3 to be devilishl...
1,2019-11-19,Four must-haves for the Tesla ‘Cybertruck’ all...
2,2019-11-20,What the Heck Is the Tesla Cybertruck? Let's W...
3,2019-11-21,Tesla all-electric ATV makes a surprise debut ...
4,2019-11-22,Tesla all-electric ATV makes a surprise debut ...


In [21]:
# Merge the newly created "ab" dataframe created above with the Stock dataframe to create a new dataframe containing the transformed data from the two dataframes into a sinle df to be loaded into porstgres for later use

combined_df = pd.merge(df,stocks_df, left_on='next_mon', right_on='Date', how='left')
combined_df

,next_mon,contents,Date,High,Low,Open,Close,Volume,Adj Close
0,2019-11-18,Tesla casually updates Model 3 to be devilishl...,2019-11-18,353.149994,346.100006,352.920013,349.989990,4400400.0,349.989990
1,2019-11-19,Four must-haves for the Tesla ‘Cybertruck’ all...,2019-11-19,359.989990,347.799988,351.750000,359.519989,7724800.0,359.519989
2,2019-11-20,What the Heck Is the Tesla Cybertruck? Let's W...,2019-11-20,361.200012,349.570007,360.000000,352.220001,6725100.0,352.220001
3,2019-11-21,Tesla all-electric ATV makes a surprise debut ...,2019-11-21,360.839996,354.000000,354.510010,354.829987,6110000.0,354.829987
4,2019-11-22,Tesla all-electric ATV makes a surprise debut ...,2019-11-22,341.000000,330.000000,340.160004,333.040009,16870600.0,333.040009
5,2019-11-25,"Tesla Cybertruck reservations hit 146,000Why t...",2019-11-25,344.570007,334.459991,344.320007,336.339996,12339500.0,336.339996
6,2019-11-26,Ford says it has nothing to prove to Tesla in ...,2019-11-26,335.500000,327.100006,335.269989,328.920013,7947400.0,328.920013
7,2019-11-27,"Tesla Cybertruck hits 250,000 pre-ordersSimone...",2019-11-27,333.929993,328.570007,331.119995,331.290009,5555600.0,331.290009
8,2019-11-28,This Tesla Cybertruck sculpted out of mashed p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2019-11-29,Tesla Cybertruck Hellcat Rendering Looks Bette...,2019-11-29,331.260010,327.500000,331.109985,329.940002,2465600.0,329.940002


In [90]:
combined_df= combined_df[['next_mon', 'contents', 'High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close']]
combined_df.rename(columns = {'next_mon':'Date', 'Adj Close':'Adj_Close'}, inplace = True)


C:\Users\danbo\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [91]:
combined_df.set_index('Date', inplace=True)
combined_df.head()

,contents,High,Low,Open,Close,Volume,Adj_Close
Date,,,,,,,
2019-11-18,Tesla casually updates Model 3 to be devilishl...,353.149994,346.100006,352.920013,349.989990,4400400.0,349.989990
2019-11-19,Four must-haves for the Tesla ‘Cybertruck’ all...,359.989990,347.799988,351.750000,359.519989,7724800.0,359.519989
2019-11-20,What the Heck Is the Tesla Cybertruck? Let's W...,361.200012,349.570007,360.000000,352.220001,6725100.0,352.220001
2019-11-21,Tesla all-electric ATV makes a surprise debut ...,360.839996,354.000000,354.510010,354.829987,6110000.0,354.829987
2019-11-22,Tesla all-electric ATV makes a surprise debut ...,341.000000,330.000000,340.160004,333.040009,16870600.0,333.040009


# Create database connection

In [95]:
# Make a connection to a Postgres SQL database

connection_string = "postgres:postgres@localhost:5432/etl_proj"
engine = create_engine(f'postgresql://{connection_string}')

In [96]:
# Confirm table name
engine.table_names()

['stockinfo']

In [101]:
combined_df.to_sql(name='stockinfo', con=engine, if_exists='append', index=True)